In [215]:
##inputs
from collections import defaultdict
contig2bactlength = defaultdict(list)

blast_file = "/mnt/projects/zilov/anna_bact/genotek_7/blast/bifid7.outfmt6"
scaffolds_file = "/mnt/projects/zilov/anna_bact/genotek_7/genotech_bifidum7.fasta"

In [216]:
def fasta_reader(fasta_file):
    fasta = {}
    header = None
    with open(fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header]="".join(seq)
                header = line
                seq = []
            else:
                seq.append(line)
    if header:
        fasta[header] = "".join(seq)
    return fasta

In [217]:
scaffolds_dir = fasta_reader(scaffolds_file)
total_length = 0
for seq in scaffolds_dir.values():
    total_length += len(seq)
print(f"Total length of scaffolds is: {total_length}")

Total length of scaffolds is: 2285457


In [218]:
## тут нужно бластить скаффолды в blast_file

In [219]:
with open(blast_file) as fh:
    for line in fh:
        line = line.strip().split("\t")
        contig = line[0]
        contig2bactlength[contig].append(line)

In [220]:
bact_percentage_in_contig = defaultdict(dict)
for contig_blast in contig2bactlength:
    value = contig2bactlength[contig_blast]
    value.sort(key=lambda x:int(x[5]), reverse=True)
    bact_seq = defaultdict(list)
    i = 0
    for blast_match in value:
        start_stop = [int(blast_match[3]), int(blast_match[4])]
        bacteria = " ".join(blast_match[-2].split()[0:2])
        if not bact_seq[bacteria]:
            bact_seq[bacteria].append(start_stop)
            #print(f"written first {bacteria}")
        else:
            #print(f"whatchin inside of {bacteria}")
            #print(f"wanna put inside {start_stop}")
            i = 1
            for seq_in in bact_seq[bacteria]:
                #print(f"will compare it to {seq_in}")
                start_in = seq_in[0]
                stop_in = seq_in[1]
                start_out = start_stop[0]
                stop_out = start_stop[1]
                #print(bact_seq)
                if start_out <= start_in and stop_out >= stop_in:
                    seq_in = start_stop
                elif (start_in <= start_out < stop_in) and stop_out > stop_in:
                    seq_in[1] = stop_out
                elif start_out < start_in and (start_in < stop_out <= stop_in):
                    seq_in[0] = start_out
                elif start_out < start_in and stop_out < start_in:
                    if i == len(bact_seq[bacteria]):
                        bact_seq[bacteria].append(start_stop)
                    else:
                        i+=1
                        continue
                elif start_out > stop_in and stop_out > stop_in:
                    if i == len(bact_seq[bacteria]):
                        bact_seq[bacteria].append(start_stop)
                    else:
                        i+=1
                        continue
                elif start_out == start_in and stop_out == stop_in:
                    continue
                elif start_out > start_in and stop_out < stop_in:
                    continue
    bact_percentage_in_contig[contig_blast] = bact_seq

In [221]:
bact2seqparts = defaultdict(dict)
for contig in bact_percentage_in_contig:
    bacteria2len = bact_percentage_in_contig[contig]
    bact_seq = defaultdict(list)
    for bacteria in bacteria2len:
        lengths = bacteria2len[bacteria]
        lengths.sort(key=lambda x:x[0], reverse=False)
        for blast_match in lengths:
            start_stop = [blast_match[0], blast_match[1]]
            if not bact_seq[bacteria]:
                bact_seq[bacteria].append(start_stop)
                #print(f"written first {bacteria} to {contig}")
            else:
                #print(f"whatchin inside of {bacteria}")
                #print(f"wanna put inside {start_stop}")
                i = 1
                for seq_in in bact_seq[bacteria]:
                    #print(f"will compare it to {seq_in}")
                    start_in = seq_in[0]
                    stop_in = seq_in[1]
                    start_out = start_stop[0]
                    stop_out = start_stop[1]
                    #print(bact_seq)
                    if start_out <= start_in and stop_out >= stop_in:
                        seq_in = start_stop
                    elif (start_in <= start_out < stop_in) and stop_out > stop_in:
                        seq_in[1] = stop_out
                    elif start_out < start_in and (start_in < stop_out <= stop_in):
                        seq_in[0] = start_out
                    elif start_out < start_in and stop_out < start_in:
                        if i == len(bact_seq[bacteria]):
                            bact_seq[bacteria].append(start_stop)
                        else:
                            i+=1
                            continue
                    elif start_out > stop_in and stop_out > stop_in:
                        if i == len(bact_seq[bacteria]):
                            bact_seq[bacteria].append(start_stop)
                        else:
                            i+=1
                            continue
                    elif start_out == start_in and stop_out == stop_in:
                        continue
                    elif start_out > start_in and stop_out < stop_in:
                        continue
    bact2seqparts[contig] = bact_seq

In [222]:
contig_content = defaultdict(dict)
for contig in bact2seqparts:
    bac_content = defaultdict(list)
    bacterias = bact2seqparts[contig]
    contig_length = len(scaffolds_dir[f">{contig}"])
    for bact in bacterias:
        bact_seqs = bacterias[bact]
        bact_length = 0
        for seq in bact_seqs:
            bact_length += seq[1] - seq[0]
        match_percent = bact_length / contig_length * 100
        bac_content[bact] = [bact_length, match_percent]
    contig_content[contig] = bac_content

In [223]:
most_common_bact = defaultdict(int)
for contig in contig_content:
    bact_content = contig_content[contig]
    for bacteria in bact_content:
        length = bact_content[bacteria]
        if length:
            most_common_bact[bacteria] += length[0]

In [224]:
percentage = 0
for bacteria in most_common_bact:
    if percentage < 99:
        length = most_common_bact[bacteria]
        bact_perc = length / total_length * 100
        percentage += bact_perc
        print(f"{bacteria} = {str(bact_perc)[:6]}%")

Bifidobacterium bifidum = 91.828%
Bifidobacterium longum = 0.2767%
Bifidobacterium breve = 4.6695%
Bifidobacterium adolescentis = 0.1616%
uncultured bacterium = 0.0359%
Bifidobacterium kashiwanohense = 0.0359%


In [225]:
for x in contig_content:
    print(x, contig_content[x])

NODE_6_length_250376_cov_52.7006_ID_7472 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [235117, 93.90556602869285]})
NODE_16_length_97245_cov_58.5053_ID_7488 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [59811, 61.505475859941384]})
NODE_18_length_90861_cov_57.3172_ID_7492 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [78190, 86.05452284258372]})
NODE_23_length_56367_cov_53.6754_ID_7502 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [56366, 99.99822591232459]})
NODE_25_length_31619_cov_55.9117_ID_7506 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [31618, 99.99683734463456]})
NODE_27_length_15494_cov_57.2561_ID_7510 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [15493, 99.99354588873112]})
NODE_28_length_14423_cov_19.51_ID_7512 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [1240, 8.597379186022325]})
NODE_31_length_5685_cov_59.1661_ID_7518 defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [5684, 99.9824098504837

In [166]:
bifidum_fasta = "/mnt/projects/zilov/anna_bact/prokaryotic_pipeline/data/bifid7/genome_assembly/spades/scaffolds_bifidum.fasta"
adolescentis_fasta = "/mnt/projects/zilov/anna_bact/prokaryotic_pipeline/data/bifid7/genome_assembly/spades/scaffolds_adolescentis.fasta"

bifidum_to_del = ["Bifidobacterium adolescentis", "Bos mutus", "Bos taurus", "Bison bonasus", "Capra hircus", "Ovis canadensis", "eukaryotic synthetic"]
adolescentis_to_del = bifidum_to_del[1:].append("Bifidobacterium bifidum")

# with open(bifidum_fasta, "w") as fw:
#     for contig, seq in scaffolds_dir.items():
#         for con, bact in contig_content.items():
#             bact_to_use = bact[]

In [124]:
def seq_finder(fasta_dict, contig, start, stop, description = "searched_seq"):
    for header, sequence in fasta_dict.items():
        if contig == header:
            print(description)
            print(sequence[int(start):int(stop)])

In [227]:
scaffolds = "/home/dzilov/data_zilov/anna_bact/prokaryotic_pipeline/data/bifid7/genome_assembly/spades/scaffolds.fasta"
scaffolds_dict = fasta_reader(scaffolds)


seq_finder(scaffolds_dict, ">NODE_7_length_236188_cov_58.0872_ID_7460", 22055, 25378, "resistance")

resistance
CACGCCTTGGCGACCTCCACACCCAGCTCGGCGCCCTCCTTCACCTCGAAGGAGGTGGCCAGCGTCTCATGGGCGATCAGATCGCGGAACCGCTCAACATCGGCCACATTGGCGGACGGCACGGTCAGCACCAGGGCGATGCGATCGGCGATGTCCAGGTCGGCGGCCTTGCGGGCGTCCTGCACGGCGCGAATCACATCGCGGGCGTAGCCCTCGGCCTCCAGGTCGGCGGTCAGCACGGTATCGAGAATCACGAAACCGCCGGTAGGCAGAGCGGCCGACACGGAAGCGTCGGCCTCGGCGGCGTTCTCCTCCTCCACGCGGTTGATTAGCTCGTATTCGCCGGCCTCCAGCGCAACCTCGCCGTTCGGCGTCTCGACGACCGGAGCACCGGTCGCGGCATCGACGTGCCAAGCACCGGTCTTGGACGCCTTGATGGCGAACTGGACCTGCTTGCCGAGGCGCGGGCCAGCGGCGCGGGCGTTGACCTTCAGCTCGTGCACGATCTTCAGCCCCTGCGAACCGGCATCCTCCATCGTGCAGAACTCAATATCCTTTATATTAAGCTCTGACTTGAGAATTTCGTCGTACGCCTTGACGGCGTCCACATCCTCGACCACGACGGTGAGCTTGGCGAGCGGCTGGCGCACACGGATCTGGGCGGCCTTGCGCAACGACAGAGTGCCGGAGACGATCTCACGCACCTTCTCCATCGCGTCCACCAGTGCCGGGTCGTCGACCAGCACACGGCCAAGCTCGGTCGCCTCACCGGTCTTCTCGTCCGAGACGTACGGCCAATCGGCCAGATGCACGGATTCGCCGCCGGTCAGGCCACGCCACACGGATTCGGACTCCATCGGGGCGAGCGGCGCGAGAACGCGCATGAACACCTCAAGCACGGTGTACAGCGTGTTGAACGCGTTCGTGTCCTCCTTCCAGAAGCGGTCACGGGTGTTGCGGATGTACCAGTTGGTGAGCACGTCGATGAA

In [179]:
for x, y in contig_content.items():
    x_len = int(x.split("_")[3])
    if x_len < 950:
        break
    for b, l in  y.items():
        if b == 0:
            continue
#         elif b in bifidum_to_del:
#             continue
        else:
            print(x)
            print(b)

NODE_1_length_716629_cov_9.319197
Bifidobacterium adolescentis
NODE_2_length_341399_cov_7.303676
Bifidobacterium adolescentis
NODE_3_length_159041_cov_54.907126
Bifidobacterium bifidum
NODE_4_length_157392_cov_10.870893
Bifidobacterium adolescentis
NODE_5_length_150296_cov_63.517637
Bifidobacterium bifidum
NODE_6_length_142245_cov_9.064883
Bifidobacterium adolescentis
NODE_7_length_139037_cov_8.863516
Bifidobacterium adolescentis
NODE_8_length_125379_cov_7.515521
Bifidobacterium adolescentis
NODE_9_length_122489_cov_10.121034
Bifidobacterium adolescentis
NODE_10_length_107442_cov_70.562587
Bifidobacterium bifidum
NODE_11_length_97565_cov_58.932480
Bifidobacterium bifidum
NODE_12_length_84447_cov_58.248245
Bifidobacterium bifidum
NODE_13_length_83012_cov_10.046607
Bifidobacterium adolescentis
NODE_14_length_82078_cov_6.470049
Bifidobacterium adolescentis
NODE_15_length_79460_cov_11.216745
Bifidobacterium adolescentis
NODE_16_length_71672_cov_61.667748
Bifidobacterium bifidum
NODE_17_len

In [198]:
adolescentis2contig = defaultdict(list)
bifidum2contig = defaultdict(list)
x2contig = defaultdict(list)


for c in contig_content:
    for b in contig_content[c]:
        if b == 0:
            continue
        elif b == "Bifidobacterium adolescentis":
            adolescentis2contig[b].append(c)
        elif b == "Bifidobacterium bifidum":
            bifidum2contig[b].append(c)
        else:
            x2contig[b].append(c)

In [201]:
def len_sum(bact2contig_dict):
    len_sum = 0
    for key in bact2contig_dict:
        for value in bact2contig_dict[key]:
            length = int(value.split("_")[3])
            len_sum+=length
    return(len_sum)


bifid_length = len_sum(bifidum2contig)
adol_len = len_sum(adolescentis2contig)
x_len = len_sum(x2contig)

print(bifid_length, adol_length, x_len)

2376494 2166893 730060


In [205]:
def extract_bact_contigs(fasta_dict, bact2contigs, new_fasta):
    with open(new_fasta, "w") as fw:
        for bact, contigs in bact2contigs.items():
            for contig in contigs:
                fasta_header = ">" + contig
                fasta_seq = fasta_dict[">" + contig]
                fw.write(f"{fasta_header}\n{fasta_seq}\n")
    print("Done!")

In [210]:
assembly = "/home/dzilov/data_zilov/anna_bact/prokaryotic_pipeline/data/bifid7/genome_assembly/spades/scaffolds.fasta"
fasta_dict = fasta_reader(assembly)
new_adol_fasta = "/home/dzilov/data_zilov/anna_bact/prokaryotic_pipeline/data/bifid7_adol/adol_clean.fasta"
new_bifid_fasta = "/home/dzilov/data_zilov/anna_bact/prokaryotic_pipeline/data/bifid7_bifid/bifid_clean.fasta"

extract_bact_contigs(fasta_dict, adolescentis2contig, new_adol_fasta)
extract_bact_contigs(fasta_dict, bifidum2contig, new_bifid_fasta)

Done!
Done!


In [214]:
first_contig_adol = "/home/dzilov/data_zilov/anna_bact/prokaryotic_pipeline/data/bifid7_adol/adol_largest_contig.fasta"
with open(first_contig_adol, "w") as fw:
    fw.write(">NODE_1_length_716629_cov_9.319197")
    fw.write(assembly_fasta_dict[">NODE_1_length_716629_cov_9.319197"])

In [232]:
bifid3_dict = fasta_reader("/mnt/projects/zilov/anna_bact/genotek_x/Bifidobacterium_bifidum_x.fna")

In [242]:
largest_contig = ">NODE_9_length_29476_cov_220.911_ID_403"
for x, y in bifid3_dict.items():
    y_len = len(y)
    if y_len > len(bifid3_dict[largest_contig]):
        largest_contig = x

In [245]:
bifid3_dict[largest_contig]

'...'

In [247]:
len("GGGGGGGGGGCGGGTGACTGGAG")

23